In [ ]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install statsmodels --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 22.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd
from scipy.integrate import quad
from itertools import product
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

**Задание 1**

In [ ]:
data = pd.read_csv('Advertising.csv', index_col=0)
data["TV*radio"] = data['TV'] * data['radio']
np.random.seed(1337)
features = ['TV', 'radio','TV*radio']
TARGET = 'sales'
model = sm.OLS(data[TARGET], sm.add_constant(data[features])).fit() 

def max_sales(costs):
  r = []
  for cost in costs:
    sum_equal_cost = LinearConstraint(np.ones(2), cost, cost)

    bounds=[(0.000001, None) for _ in range(2)]
    x0 = cost * np.ones(2) / 2

    res = minimize(lambda x: -model.predict(pd.DataFrame(
        {'const': [1], 'TV': [x[0]], 'radio': [x[1]], 'TV*radio': [x[0]*x[1]]}))[0],
                  x0=x0, bounds=bounds, constraints=[sum_equal_cost])
    r.append(res.x)
  return np.array(r)


costs = np.array(data[['TV', 'radio', 'newspaper']]).sum(axis=-1)
redist = max_sales(costs)
new_data = pd.DataFrame({'TV': redist.T[0], 'radio': redist.T[1]})
new_data['TV*radio'] = new_data['TV'] * new_data['radio']

prstd, iv_l, iv_u = wls_prediction_std(model, sm.add_constant(new_data))
variances = prstd ** 2

opt_sales = model.predict(sm.add_constant(new_data))
t = ss.t(200 - 3 - 1).ppf(0.975)
opt_sales.sum() + variances.sum()**(0.5) * np.array([-t, t])

array([4947.50579278, 5006.22824936])

**Задание 2**

In [ ]:
data = pd.read_csv('Advertising.csv', index_col=0)
data["TV*radio"] = data['TV'] * data['radio']

np.random.seed(1337)
features = ['TV', 'radio','TV*radio']
TARGET = 'sales'
model = sm.OLS(data[TARGET], sm.add_constant(data[features])).fit() 

e_star = model.resid.values
alpha = 0.05
N = 10000
n = 200
ide = np.random.choice(n, size = (N, n))
e_b = e_star[ide]

xbeta = model.fittedvalues.values

y_b = np.array([xbeta for i in range(N)]) + e_b

radio_b = []
for y in y_b:
  beta = sm.OLS(y, sm.add_constant(data[features])).fit().params.values
  radio_b.append(beta[2])

np.quantile(np.array(radio_b), alpha/2), np.quantile(np.array(radio_b), 1- alpha/2)

(0.01103169461042964, 0.04674395597269704)

**Задание 3**

$$
H_0: \gamma \in \mathbb{L}_0 ⊂ \mathbb{L}_1
$$

$$
\mathbb{E}ɛ = 0
$$

Пусть $P_0$ - проекция на $\mathbb{L}_0$, $P_1$ - проекция на $\mathbb{L}_1$

$D_0 = Y - P_0Y$

$D_{01} = P_0Y - P_1Y$

$(!) D_0 \perp D_{01}$ и $\mathbb{E}(D_0) = \mathbb{E}(D_{01}) = 0$

$$
D_0 = Y - P_0Y = \gamma + \varepsilon - P_0\gamma - P_0\varepsilon = 
\text{(т.к. $\gamma \in \mathbb{L}_0 ⊂ \mathbb{L}_1$)}
= \varepsilon - P_0\varepsilon
$$

$$
D_1 = P_0Y - P_1Y = P_0\gamma + P_0\varepsilon - P_1\gamma - P_1\varepsilon = 
\text{(т.к. $\gamma \in \mathbb{L}_0 ⊂ \mathbb{L}_1$)}
= P_0\varepsilon - P_1\varepsilon
$$

по свойству проекции: $(\varepsilon - P_0\varepsilon) \perp \mathbb{L}_0$

$P_0\varepsilon - P_1\varepsilon = P_0(\varepsilon - P_1\varepsilon) \in \mathbb{L}_0 (P_0P_1 = P_1)$.

$\Rightarrow (\varepsilon - P_0\varepsilon) \perp (P_0\varepsilon - P_1\varepsilon)$

т.е. $D_0 \perp D_{01}$

$$
\mathbb{E}(D_0) = \mathbb{E}(I - P_0)\varepsilon = (I - P_0)\mathbb{E}ɛ = 0
$$

$$
\mathbb{E}(D_1) = \mathbb{E}(P_0 - P_1)\varepsilon = (P_0 - P_1)\mathbb{E}ɛ = 0
$$



**Задание 4**

*1.*

$(!) RSS = ||D_0||^2  \sim \sigma^2 \chi^2(n - k - 1)$


$D_0 = Y - P_0Y = Y - hY$, где $h = x(x^Tx)^{-1}x^T$(hat-matrix) - оператор ортогонального проектирования на
линейную оболочку столбцов x ($\mathbb{L}_0$).

Причем, $hY = x\beta^*$

$RSS = ||Y - x\beta^*||^2$ по определению.

т.е. $RSS = ||Y - x\beta^*||^2 = ||Y - hY||^2 = ||D_0||^2$

Осталось понять почему $D_0^TD_0 \sim \sigma^2 \chi^2(n - k - 1)$

$D_0 = Y - hY = \gamma + ɛ - \gamma - hɛ = (I - h)ɛ$

$\frac{ɛ}{σ} \sim N(0, I_n)$

$(I - h)$ - проекция на оргогональное дополнение $\mathbb{L}_0$ 

$dim(\mathbb{L}_0^{\perp}) = n - k - 1$ $\Rightarrow$
$(I - h)\frac{ɛ}{σ} \sim N(0, I_{n - k - 1})$

Тогда по определению $\chi^2$:

$$
((I - h)\frac{ɛ}{σ})^T((I - h)\frac{ɛ}{σ}) \sim \chi^2(n - k - 1)
$$

$$
((I - h)ɛ)^T((I - h)ɛ) \sim \sigma^2\chi^2(n - k - 1)
$$
$$
||D_0||^2 = D_0^TD_0 = \sigma^2\chi^2(n - k - 1)
$$

*2.*

Рассмотрим базис $\mathbb{R}^n: <1, x_{:,1}, \dots, x_{:,k}> \oplus <1, x_{:,1}, \dots, x_{:,k}>^{\perp}$ и замену базиса переводящую наш базис в этот $C$.

Тогда $СhY = \beta^{*}$.

$D_0 = (I-h)Y$ и $hY$ - независимы $⇒$ $(I-h)Y$ и $СhY$ - независимы.

Значит $D_0^TD_0$ и $\beta^*$ - независимы.

*3.*

$$
\frac{c^T(\beta^* - \beta)}{\hat\sigma \sqrt{c^T(x^Tx)^{-1}c}} \sim T(n - k - 1)
$$

$$
\hat \sigma = \sqrt \frac{RSS}{n - k - 1} = \sigma \sqrt \frac{\frac{RSS}{\sigma^2}}{n - k - 1}
$$

$$
Y := \frac{RSS}{\sigma^2} \sim \chi^2(n - k - 1)
$$

$$
\beta^* - \beta = (x^Tx)^{-1}x^T\varepsilon ⇒ (\beta^*-\beta) \sim N(0, cov(\beta^*))
$$

$$
cov(\beta^*) = \sigma^2(x^Tx)^{-1}
$$

$$
\frac{c^T(\beta^* - \beta)}{\hat\sigma \sqrt{c^T(x^Tx)^{-1}c}} =
\frac{1}{\sqrt{\frac{Y}{n - k - 1}}} \frac{c^T(\beta^* - \beta)}{\sigma\sqrt{c^T(x^Tx)^{-1}c}}
$$

$$
c^T(\beta^* - \beta) \sim N(0, \sigma^2c^T(x^Tx)^{-1}c) \Rightarrow
Z := \frac{c^T(\beta^* - \beta)}{\sigma\sqrt{c^T(x^Tx)^{-1}c}} \sim N(0, 1)
$$

по предыдущим пунктам $Z$ и $Y$ - независимы.

$⇒ \frac{Z}{\sqrt\frac{Y}{n - k -1}} \sim T(n - k -1)$ по определению.
